In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# data Preprocessing
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data = data.dropna()  
    # convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    return data

#  the dataset
file_path = 'Metro-Both-Classes.csv'
data = load_and_preprocess_data(file_path)

In [2]:
data.head()


,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,class,month
0,2020-04-01 00:00:09,-0.014,8.594,8.582,-0.024,8.598,59.000,0.0450,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,2020-04
1,2020-04-01 00:00:18,-0.014,8.580,8.568,-0.024,8.584,58.950,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,2020-04
2,2020-04-01 00:00:28,-0.014,8.566,8.554,-0.024,8.570,58.800,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,2020-04
3,2020-04-01 00:00:38,-0.014,8.552,8.540,-0.022,8.556,58.700,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,2020-04
4,2020-04-01 00:00:48,-0.014,8.536,8.526,-0.024,8.540,58.575,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0,2020-04


In [3]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split


def prepare_features_and_labels(data):
    features = ['TP2', 'DV_pressure', 'Oil_temperature', 'Motor_current', 'DV_eletric', 'Towers', 'LPS', 'Oil_level', 'Caudal_impulses']
    target = 'class'

    # Select features for X and target for y
    X = data[features]
    y = data[target]
    return X, y

def scale_features(X_train, X_test=None):
    """Scale features using StandardScaler."""
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    
    if X_test is not None:
        X_test_scaled = scaler.transform(X_test)
        return X_train_scaled, X_test_scaled
    
    return X_train_scaled



In [4]:
'''def train_global_model(X_global, y_global):
    """Train initial global SVM model."""
    model = SVC(kernel='rbf')
    model.fit(X_global, y_global)
    return model
'''

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.utils import shuffle
import numpy as np

def train_global_model(X_global, y_global):
    """Train initial global SVM model using a subset of the data, balance the classes, and avoid overfitting."""
    
    # Shuffle and split data
    X, y = shuffle(X_global, y_global, random_state=42)
    X_train, X_sample, y_train, y_sample = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)
    
    # Balance classes using class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_sample), y=y_sample)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
    
    # Feature scaling
    scaler = StandardScaler()
    X_balanced = scaler.fit_transform(X_sample)
    
    # Hyperparameter tuning with StratifiedKFold
    param_grid = {
        'C': [0.1, 1, 10, 100, 1000],
        'gamma': [0.001, 0.01, 0.1, 1],
        'kernel': ['rbf', 'linear']
    }
    
    grid_search = GridSearchCV(SVC(class_weight=class_weight_dict), param_grid, cv=StratifiedKFold(5), scoring='accuracy')
    grid_search.fit(X_balanced, y_sample)
    
    # Best model from grid search
    best_model = grid_search.best_estimator_
    
    return best_model


def local_training_and_update(model, X_local, y_local):
    """Fine-tune the global model on local data."""
    model.fit(X_local, y_local)
    return model


def evaluate_model(model, X_test, y_test):
    
    y_pred = model.predict(X_test)
    print(f"Model SVM Classification Report:")
    print(classification_report(y_test, y_pred))
    print(f"Model SVM Accuracy Score:", accuracy_score(y_test, y_pred))

In [5]:
from tqdm import tqdm
import time

def federated_learning(global_data, client1_data, client2_data):
    """Perform federated learning with LSTM-CNN and track progress using tqdm."""
    
    # Prepare global data
    print("Preparing global data...")
    X_global, y_global = prepare_features_and_labels(global_data)
    X_global_scaled = scale_features(X_global)
    
    # Train global model
    print("Training global model...")
    global_model = train_global_model(X_global_scaled, y_global)
    
    # Prepare client data
    print("Preparing client data...")
    X_client1, y_client1 = prepare_features_and_labels(client1_data)
    X_client2, y_client2 = prepare_features_and_labels(client2_data)
    X_client1_scaled, X_client2_scaled = scale_features(X_client1, X_client2)
    
    # Fine-tune model on local client data
    print("Fine-tuning model on client data...")
    
    client_data = [
        (X_client1_scaled, y_client1, "Client 1"),
        (X_client2_scaled, y_client2, "Client 2")
    ]
    
    for X_client, y_client, client_name in tqdm(client_data, desc="Clients", unit="client"):
        print(f"Fine-tuning on {client_name}...")
        start_time = time.time()
        global_model = local_training_and_update(global_model, X_client, y_client)
        elapsed_time = time.time() - start_time
        print(f"Finished fine-tuning on {client_name} in {elapsed_time:.2f} seconds.")
    
    # Evaluate the updated global model
    print("Evaluating the updated global model...")
    start_time = time.time()
    evaluate_model(global_model, X_global_scaled, y_global)
    elapsed_time = time.time() - start_time
    print(f"Model evaluation completed in {elapsed_time:.2f} seconds.")
    
    print("Federated learning completed.")


In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from sklearn.utils import shuffle

def preprocess_data(data):
   
    if 'timestamp' not in data.columns or 'class' not in data.columns:
        raise ValueError("Data must contain 'timestamp' and 'class' columns.")
    
    # Ensure 'timestamp' is in datetime format
    data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')
    
    # Drop rows with invalid timestamps
    data = data.dropna(subset=['timestamp'])
    
    # Extract month as a period
    data['month'] = data['timestamp'].dt.to_period('M')
    
    # Convert periods to string for sorting
    months = data['month'].astype(str).unique()
    
    if len(months) < 4:
        raise ValueError("Not enough distinct months to split into global and client data.")
    
    months.sort()
    print(months)
    
    first_two_months = months[:2]
    last_two_months = months[-2:]
    
    # Filter data based on these months
    global_data = data[data['month'].astype(str).isin(first_two_months)]
    client1_data = data[data['month'].astype(str) == last_two_months[0]]
    client2_data = data[data['month'].astype(str) == last_two_months[1]]
    
    if global_data.empty or client1_data.empty or client2_data.empty:
        raise ValueError("One or more of the filtered datasets are empty.")
    
    return global_data, client1_data, client2_data

In [10]:

data['timestamp'] = pd.to_datetime(data['timestamp'])  # Ensure timestamp is in datetime format
global_data, client1_data, client2_data = preprocess_data(data)


['2020-04' '2020-05' '2020-06' '2020-07']


In [11]:
federated_learning(global_data, client1_data, client2_data)

Preparing global data...
Training global model...


: 

#### Final model on federated learning with attention mechanism

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import time

# Data Preprocessing
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data = data.dropna()  
    return data

def prepare_features_and_labels(data):
    features = ['TP2', 'DV_pressure', 'Oil_temperature', 'Motor_current', 'DV_eletric', 'Towers', 'LPS', 'Oil_level', 'Caudal_impulses']
    target = 'class'
    X = data[features].values
    y = data[target].values

    return X, y

def balance_and_sample(X, y, sample_fraction=0.4):
    """Sample and balance classes."""
    X, y = shuffle(X, y, random_state=42)
    sample_size = int(sample_fraction * len(X))
    X_sample, y_sample = X[:sample_size], y[:sample_size]
    
    classes = np.unique(y_sample)
    max_samples = max([np.sum(y_sample == cls) for cls in classes])
    
    X_balanced = []
    y_balanced = []
    
    for cls in classes:
        X_cls = X_sample[y_sample == cls]
        y_cls = y_sample[y_sample == cls]
        
        X_balanced.append(X_cls[:max_samples])
        y_balanced.append(y_cls[:max_samples])
    
    X_balanced = np.vstack(X_balanced)
    y_balanced = np.hstack(y_balanced)
    
    return X_balanced, y_balanced



def preprocess_data(data):
    if 'timestamp' not in data.columns or 'class' not in data.columns:
        raise ValueError("Data must contain 'timestamp' and 'class' columns.")
    
    # Ensure 'timestamp' is in datetime format
    data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')
    
    # Drop rows with invalid timestamps
    data = data.dropna(subset=['timestamp'])
    
    # Extract month as a period
    data['month'] = data['timestamp'].dt.to_period('M')
    
    # Convert periods to string for sorting
    months = data['month'].astype(str).unique()
    
    if len(months) < 4:
        raise ValueError("Not enough distinct months to split into global and client data.")
    
    months.sort()
    
    first_two_months = months[:2]
    last_two_months = months[-2:]
    
    # Filter data based on these months
    global_data = data[data['month'].astype(str).isin(first_two_months)]
    client1_data = data[data['month'].astype(str) == last_two_months[0]]
    client2_data = data[data['month'].astype(str) == last_two_months[1]]
    
    if global_data.empty or client1_data.empty or client2_data.empty:
        raise ValueError("One or more of the filtered datasets are empty.")
    
    return global_data, client1_data, client2_data

def federated_learning(global_data, client1_data, client2_data):
    """Perform federated learning with CNN-LSTM and track progress using tqdm."""
    
    # Prepare global data
    print("Preparing global data...")
    X_global, y_global = prepare_features_and_labels(global_data)
    X_global, y_global = balance_and_sample(X_global, y_global, sample_fraction=0.4)
    
    # Prepare client data
    print("Preparing client data...")
    X_client1, y_client1 = prepare_features_and_labels(client1_data)
    X_client2, y_client2 = prepare_features_and_labels(client2_data)
    X_client1, y_client1 = balance_and_sample(X_client1, y_client1, sample_fraction=0.4)
    X_client2, y_client2 = balance_and_sample(X_client2, y_client2, sample_fraction=0.4)
    
    # Reshape data for CNN-LSTM (e.g., [samples, timesteps, features])
    # Assuming 1 timestep and number of features as the second dimension
    X_global = X_global[:, np.newaxis, :]
    X_client1 = X_client1[:, np.newaxis, :]
    X_client2 = X_client2[:, np.newaxis, :]
    
    # Create and train global model
    print("Creating and training global model...")
    model = create_cnn_lstm_model(input_shape=(X_global.shape[1], X_global.shape[2]))
    model.fit(X_global, y_global, epochs=10, batch_size=32, verbose=2)
    
    # Fine-tune model on client data
    client_data = [
        (X_client1, y_client1, "Client 1"),
        (X_client2, y_client2, "Client 2")
    ]
    
    print("Fine-tuning model on client data...")
    for X_client, y_client, client_name in tqdm(client_data, desc="Clients", unit="client"):
        print(f"Fine-tuning on {client_name}...")
        start_time = time.time()
        model.fit(X_client, y_client, epochs=5, batch_size=32, verbose=2)
        elapsed_time = time.time() - start_time
        print(f"Finished fine-tuning on {client_name} in {elapsed_time:.2f} seconds.")
    
    # Evaluate the updated global model
    print("Evaluating the updated global model...")
    start_time = time.time()
    y_global_pred = (model.predict(X_global) > 0.5).astype(int)
    print("Model Classification Report:")
    print(classification_report(y_global, y_global_pred))
    print("Model Accuracy Score:", accuracy_score(y_global, y_global_pred))
    elapsed_time = time.time() - start_time
    print(f"Model evaluation completed in {elapsed_time:.2f} seconds.")
    
    print("Federated learning completed.")



In [ ]:
def create_cnn_lstm_model(input_shape):
    """Create a CNN-LSTM model."""
    inputs = Input(shape=input_shape)
    
    # CNN layers with padding to handle small sequence lengths
    x = Conv1D(filters=64, kernel_size=1, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=1)(x)
    x = Dropout(0.2)(x)
    
    # LSTM layers
    x = LSTM(50, return_sequences=True)(x)
    x = LSTM(50)(x)
    
    # Dense layers
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Example usage
file_path = 'Metro-Both-Classes.csv'
data = load_and_preprocess_data(file_path)
global_data, client1_data, client2_data = preprocess_data(data)
federated_learning(global_data, client1_data, client2_data)


Preparing global data...
Preparing client data...
Creating and training global model...
Epoch 1/10
5145/5145 - 18s - 3ms/step - accuracy: 0.9731 - loss: 0.0613
Epoch 2/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9762 - loss: 0.0451
Epoch 3/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9875 - loss: 0.0306
Epoch 4/10
5145/5145 - 13s - 2ms/step - accuracy: 0.9900 - loss: 0.0259
Epoch 5/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9920 - loss: 0.0230
Epoch 6/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9920 - loss: 0.0223
Epoch 7/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9921 - loss: 0.0224
Epoch 8/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9926 - loss: 0.0215
Epoch 9/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9928 - loss: 0.0211
Epoch 10/10
5145/5145 - 13s - 3ms/step - accuracy: 0.9924 - loss: 0.0217
Fine-tuning model on client data...


Clients:   0%|          | 0/2 [00:00<?, ?client/s]

Fine-tuning on Client 1...
Epoch 1/5
2707/2707 - 7s - 3ms/step - accuracy: 0.9969 - loss: 0.0107
Epoch 2/5
2707/2707 - 7s - 3ms/step - accuracy: 0.9974 - loss: 0.0084
Epoch 3/5
2707/2707 - 7s - 3ms/step - accuracy: 0.9979 - loss: 0.0072
Epoch 4/5
2707/2707 - 8s - 3ms/step - accuracy: 0.9982 - loss: 0.0071
Epoch 5/5
2707/2707 - 7s - 3ms/step - accuracy: 0.9979 - loss: 0.0079


Clients:  50%|█████     | 1/2 [00:35<00:35, 35.97s/client]

Finished fine-tuning on Client 1 in 35.97 seconds.
Fine-tuning on Client 2...
Epoch 1/5
2783/2783 - 7s - 3ms/step - accuracy: 0.9938 - loss: 0.0163
Epoch 2/5
2783/2783 - 7s - 3ms/step - accuracy: 0.9957 - loss: 0.0121
Epoch 3/5
2783/2783 - 7s - 3ms/step - accuracy: 0.9964 - loss: 0.0108
Epoch 4/5
2783/2783 - 7s - 3ms/step - accuracy: 0.9969 - loss: 0.0098
Epoch 5/5
2783/2783 - 7s - 3ms/step - accuracy: 0.9972 - loss: 0.0091


Clients: 100%|██████████| 2/2 [01:12<00:00, 36.12s/client]

Finished fine-tuning on Client 2 in 36.26 seconds.
Evaluating the updated global model...


5145/5145 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step
Model Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99    160227
           1       0.00      0.00      0.00      4386

    accuracy                           0.97    164613
   macro avg       0.49      0.50      0.49    164613
weighted avg       0.95      0.97      0.96    164613

Model Accuracy Score: 0.9733435390886503
Model evaluation completed in 10.68 seconds.
Federated learning completed.


# Final Model with attention mechnism and gradient compression in a CNN LSTM w Federated Learning

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Attention
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm
import time

# Data Preprocessing
def load_and_preprocess_data(file_path):
    data = pd.read_csv(file_path)
    data = data.dropna()  
    return data

def prepare_features_and_labels(data):
    features = ['TP2', 'DV_pressure', 'Oil_temperature', 'Motor_current', 'DV_eletric', 'Towers', 'LPS', 'Oil_level', 'Caudal_impulses']
    target = 'class'
    X = data[features].values
    y = data[target].values
    return X, y

def balance_and_sample(X, y, sample_fraction=0.4):
    """Sample and balance classes."""
    X, y = shuffle(X, y, random_state=42)
    sample_size = int(sample_fraction * len(X))
    X_sample, y_sample = X[:sample_size], y[:sample_size]
    
    classes = np.unique(y_sample)
    max_samples = max([np.sum(y_sample == cls) for cls in classes])
    
    X_balanced = []
    y_balanced = []
    
    for cls in classes:
        X_cls = X_sample[y_sample == cls]
        y_cls = y_sample[y_sample == cls]
        
        X_balanced.append(X_cls[:max_samples])
        y_balanced.append(y_cls[:max_samples])
    
    X_balanced = np.vstack(X_balanced)
    y_balanced = np.hstack(y_balanced)
    
    return X_balanced, y_balanced

def preprocess_data(data):
    if 'timestamp' not in data.columns or 'class' not in data.columns:
        raise ValueError("Data must contain 'timestamp' and 'class' columns.")
    
    data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce')
    data = data.dropna(subset=['timestamp'])
    data['month'] = data['timestamp'].dt.to_period('M')
    months = data['month'].astype(str).unique()
    
    if len(months) < 4:
        raise ValueError("Not enough distinct months to split into global and client data.")
    
    months.sort()
    first_two_months = months[:2]
    last_two_months = months[-2:]
    
    global_data = data[data['month'].astype(str).isin(first_two_months)]
    client1_data = data[data['month'].astype(str) == last_two_months[0]]
    client2_data = data[data['month'].astype(str) == last_two_months[1]]
    
    if global_data.empty or client1_data.empty or client2_data.empty:
        raise ValueError("One or more of the filtered datasets are empty.")
    
    return global_data, client1_data, client2_data



def compress_gradients(gradients, compression_factor=0.1):
    """Quantize gradients to reduce communication overhead."""
    compressed_gradients = {}
    for name, grad in gradients.items():
        # Quantize gradients (e.g., to 8-bit precision)
        compressed_gradients[name] = (grad / np.max(np.abs(grad)) * 127).astype(np.int8)
    return compressed_gradients




In [ ]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Multiply, Lambda
from tensorflow.keras.models import Model
import tensorflow as tf

def create_cnn_lstm_attention_model(input_shape):
    """Create a CNN-LSTM model with attention mechanism."""
    inputs = Input(shape=input_shape)
    
    # CNN layers
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2, padding='same')(x)
    x = Dropout(0.2)(x)
    
    # LSTM layers
    x = LSTM(50, return_sequences=True)(x)
    
    # Attention Mechanism
    attention_probs = Dense(x.shape[-1], activation='softmax')(x)  # Use the last dimension of x
    attention_mul = Multiply()([x, attention_probs])
    
    x = LSTM(50)(attention_mul)
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
import tensorflow as tf

def measure_communication_time(func, *args, **kwargs):
    """Measure the time taken for a function to execute."""
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    return result, elapsed_time

def compress_gradients(gradients):
    """Apply gradient compression (e.g., quantization or sparsification)."""
    compressed_gradients = []
    for grad in gradients:
        # Example of simple quantization: round gradients to the nearest integer
        compressed_gradients.append(tf.round(grad))
    return compressed_gradients

def federated_learning(global_data, client1_data, client2_data):
    """Perform federated learning with CNN-LSTM and attention mechanism, tracking communication time."""
    
    # Prepare global data
    print("Preparing global data...")
    X_global, y_global = prepare_features_and_labels(global_data)
    X_global, y_global = balance_and_sample(X_global, y_global, sample_fraction=0.4)
    
    # Prepare client data
    print("Preparing client data...")
    X_client1, y_client1 = prepare_features_and_labels(client1_data)
    X_client2, y_client2 = prepare_features_and_labels(client2_data)
    X_client1, y_client1 = balance_and_sample(X_client1, y_client1, sample_fraction=0.4)
    X_client2, y_client2 = balance_and_sample(X_client2, y_client2, sample_fraction=0.4)
    
    # Reshape data for CNN-LSTM (e.g., [samples, timesteps, features])
    X_global = X_global[:, np.newaxis, :]
    X_client1 = X_client1[:, np.newaxis, :]
    X_client2 = X_client2[:, np.newaxis, :]
    
    # Create and train global model
    print("Creating and training global model...")
    model = create_cnn_lstm_attention_model(input_shape=(X_global.shape[1], X_global.shape[2]))
    _, train_time_global = measure_communication_time(model.fit, X_global, y_global, epochs=10, batch_size=32, verbose=2)
    print(f"Time to train global model: {train_time_global:.2f} seconds.")
    
    # Fine-tune model on client data
    client_data = [
        (X_client1, y_client1, "Client 1"),
        (X_client2, y_client2, "Client 2")
    ]
    
    print("Fine-tuning model on client data...")
    for X_client, y_client, client_name in tqdm(client_data, desc="Clients", unit="client"):
        print(f"Fine-tuning on {client_name}...")
        
        # Measure time to fine-tune
        def train_func():
            model.fit(X_client, y_client, epochs=5, batch_size=32, verbose=2)
        
        _, update_time = measure_communication_time(train_func)
        print(f"Time to fine-tune on {client_name}: {update_time:.2f} seconds.")
        
        # Compress gradients and simulate communication
        with tf.GradientTape() as tape:
            # Calculate logits
            logits = model(X_client, training=True)
            
            # Ensure y_client has the correct shape
            y_client = tf.reshape(y_client, (-1, 1))
            
            # Compute loss
            loss = tf.keras.losses.binary_crossentropy(y_client, logits)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        compressed_gradients = compress_gradients(gradients)
        print(f"Compressed gradients size for {client_name}: {sum(tf.size(grad).numpy() for grad in compressed_gradients) / (1024 * 1024):.2f} MB")
    
    # Evaluate the updated global model
    print("Evaluating the updated global model...")
    y_global_pred = (model.predict(X_global) > 0.5).astype(int)
    print("Model Classification Report:")
    print(classification_report(y_global, y_global_pred))
    print("Model Accuracy Score:", accuracy_score(y_global, y_global_pred))
    
    print("Federated learning completed.")



from tensorflow.keras.layers import Bidirectional, Attention, Concatenate, Lambda
from tensorflow.keras import backend as K

def create_advanced_cnn_lstm_attention_model(input_shape):
    """Create an advanced CNN-LSTM model with an enhanced attention mechanism."""
    inputs = Input(shape=input_shape)
    
    # CNN layers
    x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2, padding='same')(x)
    x = Dropout(0.3)(x)
    
    # LSTM layers
    x = Bidirectional(LSTM(100, return_sequences=True))(x)
    
    # Attention Mechanism
    attention_probs = Dense(x.shape[-1], activation='softmax')(x)  # Use the last dimension of x
    attention_mul = Multiply()([x, attention_probs])
    
    # Further LSTM layer
    x = Bidirectional(LSTM(100))(attention_mul)
    
    # Dense layers
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model




In [ ]:
file_path = 'Metro-Both-Classes.csv'
data = load_and_preprocess_data(file_path)
global_data, client1_data, client2_data = preprocess_data(data)
federated_learning(global_data, client1_data, client2_data)

Preparing global data...
Preparing client data...
Creating and training global model...
Epoch 1/10
5145/5145 - 28s - 5ms/step - accuracy: 0.9733 - loss: 0.0661
Epoch 2/10
5145/5145 - 24s - 5ms/step - accuracy: 0.9734 - loss: 0.0487
Epoch 3/10
5145/5145 - 19s - 4ms/step - accuracy: 0.9819 - loss: 0.0382
Epoch 4/10
5145/5145 - 20s - 4ms/step - accuracy: 0.9895 - loss: 0.0277
Epoch 5/10
5145/5145 - 22s - 4ms/step - accuracy: 0.9909 - loss: 0.0246
Epoch 6/10
5145/5145 - 20s - 4ms/step - accuracy: 0.9915 - loss: 0.0231
Epoch 7/10
5145/5145 - 20s - 4ms/step - accuracy: 0.9917 - loss: 0.0230
Epoch 8/10
5145/5145 - 19s - 4ms/step - accuracy: 0.9921 - loss: 0.0225
Epoch 9/10
5145/5145 - 19s - 4ms/step - accuracy: 0.9930 - loss: 0.0205
Epoch 10/10
5145/5145 - 19s - 4ms/step - accuracy: 0.9931 - loss: 0.0202
Time to train global model: 210.52 seconds.
Fine-tuning model on client data...


Clients:   0%|          | 0/2 [00:00<?, ?client/s]

Fine-tuning on Client 1...
Epoch 1/5
2707/2707 - 9s - 3ms/step - accuracy: 0.9974 - loss: 0.0107
Epoch 2/5
2707/2707 - 9s - 3ms/step - accuracy: 0.9980 - loss: 0.0068
Epoch 3/5
2707/2707 - 9s - 3ms/step - accuracy: 0.9981 - loss: 0.0068
Epoch 4/5
2707/2707 - 9s - 3ms/step - accuracy: 0.9983 - loss: 0.0063
Epoch 5/5
2707/2707 - 13s - 5ms/step - accuracy: 0.9981 - loss: 0.0065
Time to fine-tune on Client 1: 48.37 seconds.


Clients:  50%|█████     | 1/2 [00:49<00:49, 49.69s/client]

Compressed gradients size for Client 1: 0.05 MB
Fine-tuning on Client 2...
Epoch 1/5
2783/2783 - 9s - 3ms/step - accuracy: 0.9934 - loss: 0.0191
Epoch 2/5
2783/2783 - 8s - 3ms/step - accuracy: 0.9964 - loss: 0.0113
Epoch 3/5
2783/2783 - 8s - 3ms/step - accuracy: 0.9974 - loss: 0.0081
Epoch 4/5
2783/2783 - 9s - 3ms/step - accuracy: 0.9974 - loss: 0.0081
Epoch 5/5
2783/2783 - 9s - 3ms/step - accuracy: 0.9976 - loss: 0.0085
Time to fine-tune on Client 2: 43.20 seconds.


Clients: 100%|██████████| 2/2 [01:33<00:00, 46.96s/client]

Compressed gradients size for Client 2: 0.05 MB
Evaluating the updated global model...


5145/5145 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step
Model Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99    160227
           1       0.00      0.00      0.00      4386

    accuracy                           0.97    164613
   macro avg       0.49      0.50      0.49    164613
weighted avg       0.95      0.97      0.96    164613

Model Accuracy Score: 0.9733556887973611
Federated learning completed.


c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nandi\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
